<a href="https://colab.research.google.com/github/Pikarz/data_clean_and_validation_from_olddoc_to_cardmaster/blob/main/02_optional%20-%20fix_swapped.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fix swapped cards
This notebook manages a standard that I used in my old document. If I already had a card in collection but I bought another one of the same type (same expansion, same number) that I wanted to keep in place of the former one, only one row was used to manage both cards. I used the Comments field to insert this data.

To make this particular kind of data more cohesive with the rest of the data, I used this script that decouple the two different cards.

In [ ]:
import psycopg2
import pandas as pd

def fix_swapped(db_params, data_path):
  conn = None
  curr = None
  try:
    conn = psycopg2.connect(**db_params)
    curr = conn.cursor()
  except Exception as e:
    print(f"Error while connecting to the DB: {e}")
    return

  df = pd.read_csv(data_path)
  if 'swapped_card_comment' not in df.columns:
    df['swapped_card_comment'] = ''
  for i, row in df.iterrows():
    comment = row['Comments']
    if pd.isna(comment):
      df.at[i, 'swapped_card_comment'] = 'ERROR! Comment is NaN'
      df.to_csv(data_path, index=False)
      continue
    if not comment.lower().startswith('scambiato con il nostro'):
      df.at[i, 'swapped_card_comment'] = 'ERROR! Comment does not start with "Scambiato con il nostro"'
      df.to_csv(data_path, index=False)
      continue

    old_condition = comment.lower().replace('scambiato con il nostro', '').strip().upper()
    if len(old_condition) > 3 or (len(old_condition)==3 and (old_condition[-1]!='-' and old_condition[-1]!='+')):
      df.at[i, 'swapped_card_comment'] = 'ERROR! The comment string is invalid'
      df.to_csv(data_path, index=False)
      continue

    old_selling_price = row['S-Price'] # extract old selling price
    df.at[i, 'S-Price'] = '' # save the selling price (the card is in collection now, not sold)
    old_selling_date = row['S-Date'] # same with selling date
    df.at[i, 'S-Date'] = ''
    df.at[i, 'in collection'] = True
    df.at[i, 'Comments'] = ''

    new_row = {'Card': row['Card'], 'Set': row['Set'], '#': row['#'], 'Condition': old_condition, 'Lang': row['Lang'], 'S-Price': old_selling_price, 'S-Date': old_selling_date, 'in collection': False}
    df = df.append(new_row, ignore_index=True)
    df.to_csv(data_path, index=False)

In [ ]:
data_path = 'H:/My Drive/pkmn/pok_swap.csv'

db_params = {
    'host': '192.168.1.2',
    'port': '5432',
    'database': 'cardmaster',
    'user': 'nemo',
    'password': '322322'
}

fix_swapped(db_params, data_path)

C:\Users\Anthony\AppData\Local\Temp\ipykernel_9376\2956185238.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\Anthony\AppData\Local\Temp\ipykernel_9376\2956185238.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\Anthony\AppData\Local\Temp\ipykernel_9376\2956185238.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\Anthony\AppData\Local\Temp\ipykernel_9376\2956185238.py:42: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(new_row, ignore_index=True)
C:\Users\Ant